## To submit this assignment, please solve the following problem. Then, save your notebook with the format [YOUR_NAME_HERE]\_Class10_Homework and submit via NYU Classes. 

---

OVERVIEW: Your task in this homework assingment is to continue building out our Class Project's database using the knowledge from class and files provided on NYU Classes.

### Problem 1 (4 points):

> Find the top zip codes for filming permits in 2015 and the percentages of children, adults, non-latino individuals and six_figure_income returns for these zip codes. (HINT: you should have 6 columns: zipcode, 4 percentage columns, and a filming permit count column). Submit your SQL code along with its resulting output.   _(This can be a png, jpg, pdf, screen-shot - whatever you would like)_


In [0]:
# your code here

`HINT: In SQL, you can use a "WITH" clause to give a sub-query block a name. The first few lines of this answer should be:`

```SQL 
WITH flat_irs AS (
    SELECT
        year,
        zipcode,
        SUM(CASE WHEN agi_map_id IN (1,2,3,4) THEN return_count ELSE NULL END) AS agi_under100K, 
```

In [0]:
WITH flat_irs AS (
SELECT year, irs_nyc_tax_returns.zipcode, 
    SUM(CASE WHEN agi_map_id IN (1,2,3,4) THEN return_count ELSE NULL END) AS agi_under100K, 
    SUM(CASE WHEN agi_map_id IN (5,6) THEN return_count ELSE NULL END) AS agi_over100k
FROM irs_nyc_tax_returns
GROUP BY year, irs_nyc_tax_returns.zipcode)

SELECT  nyc_film_permits.zipcode, 
100*(SUM(Age_under5) + SUM(Age_5to9) + SUM(Age_10to14) + SUM(Age_15to19)) / SUM(nyc_census_data.age_all) AS children_pct, 
100 - 100*(SUM(Age_under5) + SUM(Age_5to9) + SUM(Age_10to14) + SUM(Age_15to19)) / SUM(nyc_census_data.age_all) AS adult_pct, 
100 - 100*(SUM(Ethnicity_All_HispancLatino_Descent))/SUM(nyc_census_data.age_all) AS non_latino_pct,
100* agi_over100k / (agi_under100k + agi_over100k)   AS six_figure_income_pct,
COUNT(DISTINCT EventID) as permit_count
FROM nyc_film_permits
	INNER JOIN flat_irs
		ON flat_irs.zipcode = nyc_film_permits.zipcode
	INNER JOIN nyc_census_data
		ON nyc_census_data.zipcode = nyc_film_permits.zipcode
WHERE STRFTIME('%Y',StartDateTime) = "2015"
GROUP BY nyc_film_permits.zipcode
ORDER BY permit_count DESC
LIMIT 10

In [0]:
https://drive.google.com/open?id=1lpSEey_V579UDpXCYPhTI-Ivgoc9yyPD    #the screenshot, also uploaded with this ipynb file

---

### Problem 2 (3 points):

> What event type is most commonly seen in predominately white neighborhoods? What about in predominantly Asian neighborhoods? 

In [0]:
# your code here
SELECT EventType,COUNT(EventType) FROM
(
SELECT nyc_census_data.zipcode, EventType
FROM nyc_film_permits
	INNER JOIN nyc_census_data
		ON nyc_film_permits.zipcode = nyc_census_data.zipcode
GROUP BY nyc_census_data.zipcode
HAVING SUM(Ethnicity_White) > SUM(Ethnicity_AfricanAmerican) AND
SUM(Ethnicity_White) > SUM(Ethnicity_AfricanAmerican) AND
SUM(Ethnicity_White) > SUM(Ethnicity_NativeAmerican) AND
SUM(Ethnicity_White) > SUM(Ethnicity_Asian) AND
SUM(Ethnicity_White) > SUM(Ethnicity_PacificIslander) AND
SUM(Ethnicity_White) > SUM(Ethnicity_Other) 
)
GROUP BY EventType


In [0]:
SELECT EventType,COUNT(EventType) FROM
(
SELECT nyc_census_data.zipcode, EventType
FROM nyc_film_permits
	INNER JOIN nyc_census_data
		ON nyc_film_permits.zipcode = nyc_census_data.zipcode
GROUP BY nyc_census_data.zipcode
HAVING SUM(Ethnicity_White) > SUM(Ethnicity_AfricanAmerican) AND
SUM(Ethnicity_Asian) > SUM(Ethnicity_AfricanAmerican) AND
SUM(Ethnicity_Asian) > SUM(Ethnicity_NativeAmerican) AND
SUM(Ethnicity_Asian) > SUM(Ethnicity_White) AND
SUM(Ethnicity_Asian) > SUM(Ethnicity_PacificIslander) AND
SUM(Ethnicity_Asian) > SUM(Ethnicity_Other) 
)
GROUP BY EventType

In [0]:
https://drive.google.com/open?id=1q-39ymZhxKN0kEa_HFNmzNlWcp_JYCxR  #screenshoots
https://drive.google.com/open?id=1xIMSSb__NQbWbVsHa6LxStg-dj_5AM7o
https://drive.google.com/open?id=1CaGm7HImYojx5miqcSPULYJt-2j17MSX
# FOR white neighborhoods, shooting permit is predominant.
# for Asian neighborhoods, shooting permit is also predominant.

---

### Problem 3 (3 points):

> Find the annual count of distinct filming permits for shooting versus setup/non-shooting event types. Display each permit type grouping as its own column in the result (HINT: you should have 3 columns: year, shooting permit counts, and setup/non-shooting permit counts). Submit your SQL code along with its resulting output. (*This can be a png, jpg, pdf, screen-shot - whatever you would like*)

In [0]:
# your code here
SELECT tableA.the_year, shooting_permit_counts, NON_shooting_permit_counts FROM
(SELECT STRFTIME('%Y',StartDateTime) AS the_year, COUNT(*) AS shooting_permit_counts
FROM nyc_film_permits
WHERE EventType = "Shooting Permit"
GROUP BY the_year) tableA
,
(SELECT STRFTIME('%Y',StartDateTime) AS the_year, COUNT(*) AS NON_shooting_permit_counts
FROM nyc_film_permits
WHERE EventType != "Shooting Permit"
GROUP BY the_year) tableB
WHERE tableA.the_year = tableB.the_year

In [0]:
https://drive.google.com/open?id=1KpKCD1j5FGirn0zUkRGqTeExpEdeSw_R #the screenshot

---